In [ ]:
!pip install rut_chile
!pip install levenshtein
!pip install Unidecode
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import requests
import pandas as pd
import concurrent.futures
from tqdm import tqdm

from rut_chile import rut_chile
from unidecode import unidecode
from Levenshtein import distance

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Datos/CORFO"
results_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Resultados/CORFO"

AH004T0005596 es el archivo obtenido vía Ley de Transparencia.

In [ ]:
df_innova = pd.read_excel(os.path.join(data_dir, "AH004T0005596.xlsx"), sheet_name="Proyectos InnovaChile")
df_innova

,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario
0,1,2023,23CH-246113,Capital Humano,BENEFICIARIO / DESARROLLADOR,Nuevas estrategias de microencapsulación para ...,77131055-9,Liva Company SPA,PERSONA JURIDICA COMERCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,30000000.0,3333334.0,3333334.0
1,2,2023,23CH-246116,Capital Humano,BENEFICIARIO / DESARROLLADOR,Aprender haciendo bakeries de bajo índice glic...,77208333-5,ELAB. Y COM. CASONA EL MONTE LTDA,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,29985000.0,7059200.0,4273200.0
2,3,2023,23CH-246118,Capital Humano,BENEFICIARIO / DESARROLLADOR,Optimización de medios de cultivo sustentables...,77148405-0,LUYEF BIOTECHNOLOGIES CHILE SPA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3340000.0,3340000.0
3,4,2023,23CH-246123,Capital Humano,BENEFICIARIO / DESARROLLADOR,Producción de proteínas recombinantes a partir...,77065948-5,Pewman Innovation spa,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",28500000.0,7800000.0,5400000.0
4,5,2023,23CH-246125,Capital Humano,BENEFICIARIO / DESARROLLADOR,NGB-eDNA. Plataforma genética no-invasiva para...,77063937-9,Ecología y Genética Ambiental SpA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3500000.0,3500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9221,9222,2012,12SUP-15953,Startup (I),BENEFICIARIO / DESARROLLADOR,metrik,17404815-0,DAN STERN JUSID,NaN,NaN,20000000.0,2222222.0,2222222.0
9222,9223,2012,12SUP-15963,Startup (I),BENEFICIARIO / DESARROLLADOR,fidus writer,24095691-8,JOHANNES WILM JOHANNES WILM,NaN,NaN,20000000.0,2222222.0,2222222.0
9223,9224,2012,12SUP-15966,Startup (I),BENEFICIARIO / DESARROLLADOR,expediciones.mx,24097002-3,VALERIA PEREZ RIOS GOMEZ,NaN,NaN,20000000.0,2222222.0,2222222.0
9224,9225,2012,12SUP-15968,Startup (I),BENEFICIARIO / DESARROLLADOR,rhyboo s.a,21651777-6,JORGE ALBERTO RODRIGUEZ SUAREZ,NaN,NaN,20000000.0,2222222.0,2222222.0


In [ ]:
df_leyid = pd.read_excel(os.path.join(data_dir, "AH004T0005596.xlsx"), sheet_name="Ley I+D")
df_leyid

,id,codigo,instrumento,titulo_del_proyecto,tipo_relacion,rutdv,nombre_razon_social,monto_consolidado_ley,tipo_ctr_sii,rubro_sii
0,1,CI-1077,Ley I+D,Programa de investigación de dietas experiment...,mandante,96633780-k,MOWI CHILE S.A.,29000000,PERSONA JURIDICA COMERCIAL,B - PESCA
1,2,CI-1113,Ley I+D,Formulación de un film de plástico biodegradab...,mandante,91438000-6,CAMBIASO HNOS S A C,12720000,PERSONA JURIDICA COMERCIAL,D - INDUSTRIAS MANUFACTURERAS NO METALICAS
2,3,CI-1183,Ley I+D,Desarrollo de un sistema de tratamiento químic...,mandante,77565440-6,SOC AGROSERVICIOS ALMENDRAL LIMITADA,14024143,PERSONA JURIDICA COMERCIAL,"A - AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA"
3,4,CI-1189,Ley I+D,Investigación y desarrollo de nuevos dispositi...,mandante,79975550-5,LIVENTUS S.A.,236959925,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU..."
4,5,CI-1303,Ley I+D,Desarrollo de un Modelo de Operación de Sistem...,mandante,76218856-2,COLBUN TRANSMISION SA,39065794,PERSONA JURIDICA COMERCIAL,"F - SUMINISTRO DE ELECTRICIDAD, GAS Y AGUA"
...,...,...,...,...,...,...,...,...,...,...
1340,1344,PI-998,Ley I+D,Eficacia y toxicidad del uso de una familia de...,mandante,77424780-7,EWOS CHILE ALIMENTOS LIMITADA,76835703,PERSONA JURIDICA COMERCIAL,D - INDUSTRIAS MANUFACTURERAS NO METALICAS
1341,1345,PI-99869,Ley I+D,Análisis Proteómico por Machine-Learning: Perf...,mandante,76221912-3,LABORATORIO PATHOVET LIMITADA,23076375,PERSONA JURIDICA COMERCIAL,O - SERVICIOS SOCIALES Y DE SALUD
1342,1346,PI-99871,Ley I+D,Desarrollo de un proceso de recubrimiento o lí...,mandante,76082953-6,NANOTEC S.A.,19770000,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU..."
1343,1347,PI-99872,Ley I+D,Nuevos usos de nanopartículas metálicas: Desar...,mandante,76082953-6,NANOTEC S.A.,19800000,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU..."


In [ ]:
df_corfo = pd.concat((df_innova, df_leyid), ignore_index=True)
df_corfo

,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley
0,1,2023.0,23CH-246113,Capital Humano,BENEFICIARIO / DESARROLLADOR,Nuevas estrategias de microencapsulación para ...,77131055-9,Liva Company SPA,PERSONA JURIDICA COMERCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,30000000.0,3333334.0,3333334.0,NaN
1,2,2023.0,23CH-246116,Capital Humano,BENEFICIARIO / DESARROLLADOR,Aprender haciendo bakeries de bajo índice glic...,77208333-5,ELAB. Y COM. CASONA EL MONTE LTDA,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,29985000.0,7059200.0,4273200.0,NaN
2,3,2023.0,23CH-246118,Capital Humano,BENEFICIARIO / DESARROLLADOR,Optimización de medios de cultivo sustentables...,77148405-0,LUYEF BIOTECHNOLOGIES CHILE SPA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3340000.0,3340000.0,NaN
3,4,2023.0,23CH-246123,Capital Humano,BENEFICIARIO / DESARROLLADOR,Producción de proteínas recombinantes a partir...,77065948-5,Pewman Innovation spa,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",28500000.0,7800000.0,5400000.0,NaN
4,5,2023.0,23CH-246125,Capital Humano,BENEFICIARIO / DESARROLLADOR,NGB-eDNA. Plataforma genética no-invasiva para...,77063937-9,Ecología y Genética Ambiental SpA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3500000.0,3500000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10566,1344,NaN,PI-998,Ley I+D,mandante,Eficacia y toxicidad del uso de una familia de...,77424780-7,EWOS CHILE ALIMENTOS LIMITADA,PERSONA JURIDICA COMERCIAL,D - INDUSTRIAS MANUFACTURERAS NO METALICAS,NaN,NaN,NaN,76835703.0
10567,1345,NaN,PI-99869,Ley I+D,mandante,Análisis Proteómico por Machine-Learning: Perf...,76221912-3,LABORATORIO PATHOVET LIMITADA,PERSONA JURIDICA COMERCIAL,O - SERVICIOS SOCIALES Y DE SALUD,NaN,NaN,NaN,23076375.0
10568,1346,NaN,PI-99871,Ley I+D,mandante,Desarrollo de un proceso de recubrimiento o lí...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19770000.0
10569,1347,NaN,PI-99872,Ley I+D,mandante,Nuevos usos de nanopartículas metálicas: Desar...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19800000.0


## Verificar ruts válidos

In [ ]:
def valid_rut_wrapper(input):
  if type(input) != str:
    return False
  elif not (input[0].isdigit() and (input[-1].isdigit() or input[-1].lower() == "k")):
    return False
  else:
    return rut_chile.is_valid_rut(input)

df_corfo["rut_normalizado"] = df_corfo["rutdv"].str.lower().str.strip().str.replace(" ", "")
df_corfo["rut_valido"] = df_corfo["rut_normalizado"].apply(valid_rut_wrapper)

In [ ]:
df_corfo[~df_corfo["rut_valido"]]

,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido
187,188,2023.0,23EURE-245956,Crea y Valida Eureka,ENTIDAD COLABORADORA,Desarrollo y evaluación de un proceso para la ...,NaN,Universidad de Vigo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
188,189,2023.0,23EURE-245956,Crea y Valida Eureka,ENTIDAD COLABORADORA,Desarrollo y evaluación de un proceso para la ...,NaN,ARTICA INGENIERÍA E INNOVACIÓN S.L.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
196,197,2023.0,23IAT-246199,Alta tecnología,ASOCIADO,Escalamiento y validación del proceso de recup...,NaN,"Ecometales Limited, Agencia en Chile",SOCIEDADES EXTRANJERAS,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,NaN,NaN,NaN,NaN,NaN,False
480,481,2022.0,22CV-206968,Crea y Valida Rápida Implementación,ENTIDAD COLABORADORA,Mejora en el proceso de descontaminación de m...,NaN,Ludlum Measurements INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
527,528,2022.0,22CV-207273,Crea y Valida Rápida Implementación,ENTIDAD COLABORADORA,Desarrollo y validación de un novedoso snack r...,NaN,Centro Tecnológico para la innovación en alime...,ORG. SIN FINES DE LUCRO,OTRAS ACTIVIDADES DE SERVICIOS,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7783,7784,2014.0,14BPCR-33459,Bienes Públicos Para La Competitividad,COEJECUTOR / CODESARROLLADOR,mapeo eolico con generacion net billing en sis...,NaN,HOHHOT BRANCH CHINESSE ACADEMY OF AGRICULTURAL...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8143,8144,2013.0,13BPC3-19021,Bienes Públicos Para La Competitividad,COEJECUTOR / CODESARROLLADOR,identificar elementos de valor en residuos min...,NaN,GEOWISSENSCHAFTEN UND ROHSTOFFE (BGR),NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8491,8492,2012.0,12BPC2-13453,Bienes Públicos Para La Competitividad,COEJECUTOR / CODESARROLLADOR,criterios para estandarizar la caracterización...,NaN,SINTEF FISKERI OG HAVBRUK AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
8496,8497,2012.0,12BPC2-13470,Bienes Públicos Para La Competitividad,COEJECUTOR / CODESARROLLADOR,desarrollo del arca del gusto en chile bases p...,NaN,FUNDACION SLOW FOOD PARA LA BIODIVERSIDAD ONLUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


Hay 83 filas con ruts inválidos

## Normalizar nombres

In [ ]:
def wrap_encoder(text):
  if type(text) != str:
    return text
  return unidecode(text)

df_corfo["nombre_normalizado"] = df_corfo["nombre_razon_social"].str.upper().str.strip().str.replace(".", "").str.replace("  ", " ").apply(wrap_encoder)
df_corfo

<ipython-input-23-f1a67afb4dea>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_corfo["nombre_normalizado"] = df_corfo["nombre_razon_social"].str.upper().str.strip().str.replace(".", "").str.replace("  ", " ").apply(wrap_encoder)


,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido,nombre_normalizado
0,1,2023.0,23CH-246113,Capital Humano,BENEFICIARIO / DESARROLLADOR,Nuevas estrategias de microencapsulación para ...,77131055-9,Liva Company SPA,PERSONA JURIDICA COMERCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,30000000.0,3333334.0,3333334.0,NaN,77131055-9,True,LIVA COMPANY SPA
1,2,2023.0,23CH-246116,Capital Humano,BENEFICIARIO / DESARROLLADOR,Aprender haciendo bakeries de bajo índice glic...,77208333-5,ELAB. Y COM. CASONA EL MONTE LTDA,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,29985000.0,7059200.0,4273200.0,NaN,77208333-5,True,ELAB Y COM CASONA EL MONTE LTDA
2,3,2023.0,23CH-246118,Capital Humano,BENEFICIARIO / DESARROLLADOR,Optimización de medios de cultivo sustentables...,77148405-0,LUYEF BIOTECHNOLOGIES CHILE SPA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3340000.0,3340000.0,NaN,77148405-0,True,LUYEF BIOTECHNOLOGIES CHILE SPA
3,4,2023.0,23CH-246123,Capital Humano,BENEFICIARIO / DESARROLLADOR,Producción de proteínas recombinantes a partir...,77065948-5,Pewman Innovation spa,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",28500000.0,7800000.0,5400000.0,NaN,77065948-5,True,PEWMAN INNOVATION SPA
4,5,2023.0,23CH-246125,Capital Humano,BENEFICIARIO / DESARROLLADOR,NGB-eDNA. Plataforma genética no-invasiva para...,77063937-9,Ecología y Genética Ambiental SpA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",30000000.0,3500000.0,3500000.0,NaN,77063937-9,True,ECOLOGIA Y GENETICA AMBIENTAL SPA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10566,1344,NaN,PI-998,Ley I+D,mandante,Eficacia y toxicidad del uso de una familia de...,77424780-7,EWOS CHILE ALIMENTOS LIMITADA,PERSONA JURIDICA COMERCIAL,D - INDUSTRIAS MANUFACTURERAS NO METALICAS,NaN,NaN,NaN,76835703.0,77424780-7,True,EWOS CHILE ALIMENTOS LIMITADA
10567,1345,NaN,PI-99869,Ley I+D,mandante,Análisis Proteómico por Machine-Learning: Perf...,76221912-3,LABORATORIO PATHOVET LIMITADA,PERSONA JURIDICA COMERCIAL,O - SERVICIOS SOCIALES Y DE SALUD,NaN,NaN,NaN,23076375.0,76221912-3,True,LABORATORIO PATHOVET LIMITADA
10568,1346,NaN,PI-99871,Ley I+D,mandante,Desarrollo de un proceso de recubrimiento o lí...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19770000.0,76082953-6,True,NANOTEC SA
10569,1347,NaN,PI-99872,Ley I+D,mandante,Nuevos usos de nanopartículas metálicas: Desar...,76082953-6,NANOTEC S.A.,PERSONA JURIDICA COMERCIAL,"H - COMERCIO AL POR MAYOR Y MENOR, REP. VEH.AU...",NaN,NaN,NaN,19800000.0,76082953-6,True,NANOTEC SA


### Arreglar RUTs

In [ ]:
df_corfo.loc[(~df_corfo["rut_valido"]) & (~df_corfo["rut_normalizado"].isnull())][["nombre_normalizado", "rut_normalizado"]]

,nombre_normalizado,rut_normalizado
1899,CYCLO,52215526652-8
2485,UNIVERSIDAD DE SANTIAGO DE CHILE,90911000-7
3312,GERHARD HUBER,extranjer-a
3317,MEDXCELL SAX,che49886772-8
3514,ECONCIENCIA INTERNACIONAL SAS,901162973-4
3515,"ZHANGJIAGANG HORSE RIDER MACHINERY CO, LTD",extranjer-a
3588,MERKEN BIOTECH SPA,766437125-
3616,INNERVYCS SPA,762491834-
3636,SERCOIN LTDA NOGALES,781751200-
3640,MOVENER SPA,766080251-


In [ ]:
to_fix = [(1899, "76506085-0"),
          (2485, "60911000-7"),
          (3588, "76643712-5"),
          (3616, "76249183-4"),
          (3636, "78175120-0"),
          (3640, "76608025-1"),
          (4178, "61980520-8"),
          (7517, "76419769-0")]

for (index, rut) in to_fix:
  df_corfo.loc[index, ["rut_normalizado", "rut_valido"]] = (rut, True)

df_corfo[df_corfo.index.isin([idx for idx, _ in to_fix])]

,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido,nombre_normalizado
1899,1900,2021.0,21SEC-171198,Súmate a innovar Economía circular,ENTIDAD COLABORADORA,Incorporación de KPIs ambientales y climáticos...,52215526652-8,Cyclo,NaN,NaN,NaN,NaN,NaN,NaN,76506085-0,True,CYCLO
2485,2486,2020.0,20CVCEK-126743,Crea y valida eureka,ENTIDAD COLABORADORA,ScanWater: Monitoreo de calidad en agua superc...,90911000-7,Universidad de Santiago de Chile,NaN,NaN,NaN,NaN,NaN,NaN,60911000-7,True,UNIVERSIDAD DE SANTIAGO DE CHILE
3588,3589,2019.0,19SN-112305,Súmate a Innovar,ENTIDAD COLABORADORA,producción escala piloto realización de prueba...,766437125-,merken biotech spa,NaN,NaN,NaN,NaN,NaN,NaN,76643712-5,True,MERKEN BIOTECH SPA
3616,3617,2019.0,19SN-112375,Súmate a Innovar,ENTIDAD COLABORADORA,msmap: optimización de riego basado en modelo ...,762491834-,Innervycs SpA,NaN,NaN,NaN,NaN,NaN,NaN,76249183-4,True,INNERVYCS SPA
3636,3637,2019.0,19SN-112421,Súmate a Innovar,ENTIDAD COLABORADORA,pfeifer caving system,781751200-,SERCOIN Ltda Nogales,NaN,NaN,NaN,NaN,NaN,NaN,78175120-0,True,SERCOIN LTDA NOGALES
3640,3641,2019.0,19SN-112437,Súmate a Innovar,ENTIDAD COLABORADORA,transformación de la matriz energética de los ...,766080251-,Movener SpA,NaN,NaN,NaN,NaN,NaN,NaN,76608025-1,True,MOVENER SPA
4178,4179,2018.0,18ISV-93396,Línea 2: Validación Y Empaquetamiento De Innov...,COEJECUTOR / CODESARROLLADOR,uso de lana de oveja del como aislante termico,51980520-8,Universidad de Aysén,NaN,NaN,NaN,NaN,NaN,NaN,61980520-8,True,UNIVERSIDAD DE AYSEN
7517,7518,2015.0,15VEIID-45670,Línea 2: Validación Y Empaquetamiento De Innov...,ASOCIADO,optimizacion de extractos vegetales en su pote...,74419769-0,FISH VET GROUP spa,NaN,NaN,NaN,NaN,NaN,NaN,76419769-0,True,FISH VET GROUP SPA


El resto de los ruts será "-"

In [ ]:
df_corfo.loc[(~df_corfo["rut_valido"]) & (~df_corfo["rut_normalizado"].isnull()),  "rut_normalizado"] = "-"

Hay RUTs que están con formato distinto, sin guión.

In [ ]:
def wrapper_format_rut(text):
  try:
    return rut_chile.format_rut_without_dots(text)
  except:
    return text

df_corfo["rut_formato"] = df_corfo["rut_normalizado"].apply(wrapper_format_rut)
df_corfo[(df_corfo["rut_formato"] != df_corfo["rut_normalizado"]) & (~df_corfo["rut_formato"].isnull())]

,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido,nombre_normalizado,rut_formato
10364,1141,NaN,PI-5116,Ley I+D,mandante,Entel Ocean,925800007,EMPRESA NACIONAL DE TELECOMUNICACIONES S.A.,PERSONA JURIDICA COMERCIAL,INFORMACION Y COMUNICACIONES,NaN,NaN,NaN,2.053269e+09,925800007,True,EMPRESA NACIONAL DE TELECOMUNICACIONES SA,92580000-7
10394,1171,NaN,PI-5161,Ley I+D,mandante,Nuevos desarrollos farmacéuticos y analíticos ...,896888005,KNOP LABORATORIOS S.A.,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,NaN,NaN,NaN,2.061496e+09,896888005,True,KNOP LABORATORIOS SA,89688800-5
10395,1172,NaN,PI-5173,Ley I+D,mandante,Sistema de Seguimiento Solar Fotovoltaico de A...,968422006,INOXCENTRO COMERCIAL S.A.,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,NaN,NaN,NaN,1.202341e+08,968422006,True,INOXCENTRO COMERCIAL SA,96842200-6
10396,1173,NaN,PI-5176,Ley I+D,mandante,Nueva tecnología de cultivo para la especie Ch...,969299607,ORIZON S.A.,PERSONA JURIDICA COMERCIAL,INDUSTRIA MANUFACTURERA,NaN,NaN,NaN,4.865086e+08,969299607,True,ORIZON SA,96929960-7
10397,1174,NaN,PI-5181,Ley I+D,mandante,Desarrollo de protocolos de manejo de cultivos...,836938003,MONSANTO CHILE S.A.,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",NaN,NaN,NaN,3.530008e+08,836938003,True,MONSANTO CHILE SA,83693800-3
10398,1175,NaN,PI-5182,Ley I+D,mandante,"Rubisco SpA, Producción de Ingredientes activo...",762766884,RUBISCO BIOTECHNOLOGY SPA,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",NaN,NaN,NaN,4.032890e+07,762766884,True,RUBISCO BIOTECHNOLOGY SPA,76276688-4
10399,1176,NaN,PI-5190,Ley I+D,mandante,Establecimiento de un programa de mejoramiento...,765517699,VIVEROS FRUTURA SPA,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",NaN,NaN,NaN,2.490235e+08,765517699,True,VIVEROS FRUTURA SPA,76551769-9
10401,1178,NaN,PI-5217,Ley I+D,mandante,CENTRO DE INVESTIGACIÓN E INNOVACIÓN DE LA FRU...,782951505,AGRICOLA LLAHUEN S.A.,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",NaN,NaN,NaN,3.505174e+09,782951505,True,AGRICOLA LLAHUEN SA,78295150-5
10402,1179,NaN,PI-5239,Ley I+D,mandante,Diseño e implementación de una estrategia de o...,965687408,GASCO GLP S.A.,PERSONA JURIDICA COMERCIAL,"SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ...",NaN,NaN,NaN,7.226560e+08,965687408,True,GASCO GLP SA,96568740-8
10404,1181,NaN,PI-5240,Ley I+D,mandante,Desarrollo de Modelo de Customer Lifetime Valu...,941410006,DERCO S.A.,PERSONA JURIDICA COMERCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,NaN,NaN,NaN,1.006416e+09,941410006,True,DERCO SA,94141000-6


In [ ]:
df_corfo["rut_normalizado"] = df_corfo["rut_formato"]
df_corfo.drop(columns=["rut_formato"], inplace=True)

Un rut por nombre

In [ ]:
for name in df_corfo["nombre_normalizado"].unique():
  ruts = df_corfo.loc[df_corfo["nombre_normalizado"] == name, "rut_normalizado"].unique()
  if len(ruts) != 1:
    print(name, ruts, len(ruts))

UNIVERSIDAD DE CONCEPCION ['81494400-k' '11809672-k'] 2
nan [] 0
FUTURE FOOD SPA ['76945848-4' '76923566-3'] 2
FRACTAL SPA ['77020592-1' '77053415-1'] 2
AEX GROUP SPA [nan '76055840-0'] 2
DESERT KING CHILE SA ['77510780-4' '96768300-0'] 2
ROBOTEC SA ['96927110-9' '76981618-6'] 2
PATAFOODS ['76200601-4' '8556557-5'] 2


In [ ]:
to_fix = [("UNIVERSIDAD DE CONCEPCION", "81494400-k"),
          ("FUTURE FOOD SPA", "76945848-4"),
          ("FRACTAL SPA", "77053415-1"),
          ("AEX GROUP SPA", "76055840-0"),
          ("DESERT KING CHILE SA", "96768300-0"),
          ("ROBOTEC SA", "76981618-6"),
          ("PATAFOODS", "76200601-4")]

for (name, rut) in to_fix:
  df_corfo.loc[df_corfo["nombre_normalizado"] == name, "rut_normalizado"] = rut

df_corfo[df_corfo["nombre_normalizado"].isin([name for name, _ in to_fix])]

,id,año_adjudicacion,codigo,instrumento,tipo_relacion,titulo_del_proyecto,rutdv,nombre_razon_social,tipo_ctr_sii,rubro_sii,aprobado_corfo,aprobado_privado_total,aprobado_privado_pecuniario,monto_consolidado_ley,rut_normalizado,rut_valido,nombre_normalizado
34,35,2023.0,23CVC-241444,Crea y Valida I+D+i Colaborativo,ENTIDAD COLABORADORA,Espuma aislante e ignífuga para la construcción,81494400-K,Universidad de Concepción,ORG. SIN FINES DE LUCRO,ENSENANZA,NaN,NaN,NaN,NaN,81494400-k,True,UNIVERSIDAD DE CONCEPCION
38,39,2023.0,23CVC-241482,Crea y Valida I+D+i Colaborativo,ENTIDAD COLABORADORA,Proceso de dosificación de agente potabilizado...,81494400-K,Universidad de Concepción,ORG. SIN FINES DE LUCRO,ENSENANZA,NaN,NaN,NaN,NaN,81494400-k,True,UNIVERSIDAD DE CONCEPCION
41,42,2023.0,23CVC-241516,Crea y Valida I+D+i Colaborativo,ENTIDAD COLABORADORA,Plug and Play Motors: Tecnología motriz eficie...,81494400-K,Universidad de Concepcion,ORG. SIN FINES DE LUCRO,ENSENANZA,NaN,NaN,NaN,NaN,81494400-k,True,UNIVERSIDAD DE CONCEPCION
48,49,2023.0,23CVC-245562,Crea y Valida I+D+i Colaborativo,ENTIDAD COLABORADORA,Reciclaje textil para el desarrollo de recubri...,81494400-K,Universidad de Concepción,ORG. SIN FINES DE LUCRO,ENSENANZA,NaN,NaN,NaN,NaN,81494400-k,True,UNIVERSIDAD DE CONCEPCION
74,75,2023.0,23CVC-245747,Crea y Valida I+D+i Colaborativo,ENTIDAD COLABORADORA,Resina adhesiva en base a polifenoles naturales,81494400-K,Universidad de Concepción,ORG. SIN FINES DE LUCRO,ENSENANZA,NaN,NaN,NaN,NaN,81494400-k,True,UNIVERSIDAD DE CONCEPCION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9672,447,NaN,PI-2180,Ley I+D,mandante,1) Desarrollo de un proceso de producción de c...,96768300-0,DESERT KING CHILE S.A.,PERSONA JURIDICA COMERCIAL,"A - AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA",NaN,NaN,NaN,3.482822e+08,96768300-0,True,DESERT KING CHILE SA
9830,605,NaN,PI-2769,Ley I+D,asociado,Programa de Innovación y Desarrollo Tecnológic...,76055840-0,AEX GROUP SPA,PERSONA JURIDICA COMERCIAL,"L - ACTIVIDADES INMOBILIARIAS, EMPRESARIALES Y...",NaN,NaN,NaN,2.915304e+08,76055840-0,True,AEX GROUP SPA
10228,1003,NaN,PI-4465,Ley I+D,mandante,desarrollo de nuevas variedades clonales de qu...,96768300-0,DESERT KING CHILE S.A.,PERSONA JURIDICA COMERCIAL,"AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",NaN,NaN,NaN,8.063456e+09,96768300-0,True,DESERT KING CHILE SA
10247,1022,NaN,PI-4506,Ley I+D,mandante,sistema de encajonado de alta velocidad multif...,76981618-6,ROBOTEC S.A.,PERSONA JURIDICA COMERCIAL,"ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",NaN,NaN,NaN,2.556665e+08,76981618-6,True,ROBOTEC SA


In [ ]:
for name in df_corfo["nombre_normalizado"].unique():
  ruts = df_corfo.loc[df_corfo["nombre_normalizado"] == name, "rut_normalizado"].unique()
  if len(ruts) != 1:
    print(name, ruts, len(ruts))

nan [] 0


In [ ]:
df_corfo.to_excel(os.path.join(results_dir, "data_corfo_ruts_imputados.xlsx"))